In [ ]:
# default_exp getUser

# SignIn

> Log into account

In [ ]:
#hide
import src.setEnv 

In [ ]:
#export 
from src.userTable import UserTable
from awsSchema.apigateway import Response, Event
from copy import deepcopy
from src.saltHashPassword import hash_password, check_password
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from beartype import beartype

## Sign In Input Class

In [ ]:
#export 
@dataclass_json
@dataclass
class SignInInput:
    username: str
    password: str
    
    @property
    def passwordHash(self):
        return hash_password(self.password)
    
    def checkPassword(self):
        passwordHashed = self.passwordHash
        

NameError: name 'dataclasses_json' is not defined

## Main Lambda Handler

In [ ]:
#export
class H:
    class ParseInputError(Exception): pass
    @staticmethod
    @beartype
    def parseInput(event: dict)->SignInInput:
        try:
            user = Event.parseDataClass(SignInInput, deepcopy(event))
            return user
        except Exception as e:
            raise ParseInputError(e)
        
    # def 

In [ ]:
#export
def signIn(event, *args):
    # user = H.parseInput(event)
    # userId = user.username
    pass